In [1]:
import os
import sys
sys.path.append('../src-py/')
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from datasets import load_dataset, load_metric, Dataset
import torch
from project_debater_api import *
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
tokenizer = AutoTokenizer.from_pretrained("../../../data-ceph/arguana/arg-generation/conclusion-generation-models/dbart")
model = AutoModelForSeq2SeqLM.from_pretrained("../../../data-ceph/arguana/arg-generation/conclusion-generation-models/dbart").to(device)

In [6]:
valid_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [7]:
valid_df[['masked_premises', 'title']].head()

,masked_premises,title
295914,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.]",Pink is just light red and doesn't deserve it's own name.
295915,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.]",Pink is just light red and doesn't deserve it's own name.
295916,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.]",Pink is just light red and doesn't deserve it's own name.
295917,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.]",Pink is just light red and doesn't deserve it's own name.
295918,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.]",Pink is just light red and doesn't deserve it's own name.


#### Mining important concepts from posts:

In [8]:
post = ['while i ve been pro choice for as long as i have been aware of the existence of abortion, i can somewhat sympathize with elements of the pro life mindset.',
 'after all, if you truly believed that a fertilized egg is the same thing as a baby, then naturally you d regard abortion as child murder, and being angry hateful towards perceived murderers of children is pretty understandable.',
 'under that mindset, taking political action to ensure as few abortions as possible is sensible and relatable.',
 'where i lose that sympathy is in seeing the sum total of their political goals, related back to the idea that abortion is essentially child murder.',
 'for example, expanded access to contraceptives and comprehensive sex education are both shown to reduce the rates of both unintended pregnancy and abortion.',
 'if i truly believed that every abortion was a murdered baby, one could reasonably expect me to support pretty much any policy that would decrease the number of abortions, including these two.',
 'yet most of the same people who equate abortion with murder also support policies that restrict sex education and access to contraceptives, which go against the fundamental idea of preventing abortions.',
 'my belief is that pro lifers want fewer abortions only in the context of an overall socially conservative atmosphere, of which fewer abortions is just one small part.',
 'i believe that this worldview is more concerned with not being the sort of society where on demand abortion is legal than it is with the actual abortions themselves.']

In [28]:
%autoreload
from project_debater_api import *

In [48]:
def get_wiki_concepts(posts, min_len=1, max_concepts=3):
    post_concepts = []
    for post in posts:
        doc = nlp(' '.join(post)) if type(post) == list else nlp(post)
        ncs = [np.text for np in doc.noun_chunks]
        wiki_concepts = term_wikifier(ncs)
        wiki_concepts = filter(lambda x: len(x[0].split()) > 1, wiki_concepts)
        wiki_concepts = sorted(list(set(wiki_concepts)),key=lambda x: -x[1])
        #take top max_concepts
        post_concepts.append([x[0] for x in wiki_concepts[:max_concepts]])
        
    return post_concepts

In [41]:
post_nps = get_wiki_concepts([post])

TermWikifierClient: 100%|██████████| 71/71 [00:00<00:00, 175.08it/s]

In [42]:
post_nps

[[('Birth control', 2159),
  ('Social conservatism', 1716),
  ('Anti-abortion movement', 1613)]]

TermWikifierClient: 100%|██████████| 71/71 [00:10<00:00, 175.08it/s]

### Generate multiple conclusions:

In [62]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":20,
    "top_p":0.95, 
    "num_beams":1
}

In [63]:
from transformers.generation_logits_process import * 

class PrefixConstrainedLogitsProcessor(LogitsProcessor):
    r"""
    [`LogitsProcessor`] that enforces constrained generation and is useful for prefix-conditioned constrained
    generation. See [Autoregressive Entity Retrieval](https://arxiv.org/abs/2010.00904) for more information.

    Args:
        prefix_allowed_tokens_fn: (`Callable[[int, torch.Tensor], List[int]]`):
            This function constraints the beam search to allowed tokens only at each step. This function takes 2
            arguments `inputs_ids` and the batch ID `batch_id`. It has to return a list with the allowed tokens for the
            next generation step conditioned on the previously generated tokens `inputs_ids` and the batch ID
            `batch_id`.
    """

    def __init__(self, prefix_allowed_tokens_fn: Callable[[int, torch.Tensor], List[int]], num_beams: int):
        self._prefix_allowed_tokens_fn = prefix_allowed_tokens_fn
        self._num_beams = num_beams

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        mask = torch.full_like(scores, -math.inf)
        for batch_id, beam_sent in enumerate(input_ids.view(-1, self._num_beams, input_ids.shape[-1])):
            for beam_id, sent in enumerate(beam_sent):
                enforced_vocab = self._prefix_allowed_tokens_fn(batch_id, sent)
                if enforced_vocab != None:
                    mask[batch_id * self._num_beams + beam_id, enforced_vocab] = 0
                else:
                    mask[batch_id * self._num_beams + beam_id, :] = 0

        return scores + mask

def allowed_prefix(prefixs, tokenizer, batch_id, sent):
    #print(batch_id)
    #print(sent)
    prefix     = prefixs[batch_id]
    gen_tokens = sent[1:]
    encoded_pref = tokenizer.encode(prefix, add_special_tokens=False)
    decoded_sent = tokenizer.decode(gen_tokens)
    
    remained_tokens = [x for x in encoded_pref if x not in gen_tokens]
    
    if len(remained_tokens) > 0: #just control the first token
        #print('rest: ', remained_tokens)
        return remained_tokens
    else:
        None

In [60]:
def generate_conclusion(premises, prefixs, gen_kwargs, batch_size=16):
    
    processors = LogitsProcessorList()
    processors.append(PrefixConstrainedLogitsProcessor(
        lambda batch_id, sent: allowed_prefix(prefixs, tokenizer, batch_id, sent), gen_kwargs['num_beams'] // 1))

    if type(premises[0]) == list:
        premises = [' '.join(x) for x in premises]
    
    ds = Dataset.from_dict({'premises': premises})
    ds = ds.map(lambda x :tokenizer(x['premises'], max_length=512, truncation=True, padding='max_length') , batched=True)
    ds.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)

    generated_conclusion = []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            generated_tokens = model.generate(
                input_ids,
                attention_mask=attention_mask,
                logits_processor = processors,
                **gen_kwargs
            )

            generated_tokens = generated_tokens.cpu().numpy()

            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]

            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

            generated_conclusion += decoded_preds

    return generated_conclusion

In [27]:
#valid_df[valid_df.post.str.len() < 50][['post', 'title']].sample(10).head()

In [28]:
#sample_posts = valid_df[valid_df.post.str.len() < 50].sample(10)['post'].tolist()

In [43]:
sample_posts = post

In [41]:
generate_conclusion(sample_posts[:3],['Gun control ', 'It is', 'He is'], gen_kwargs)

  0%|          | 0/1 [00:00<?, ?ba/s]

['Gun control  is an sensible and relatable viewpoint',
 'It is important to preserve the artistic integrity of great works of art like Huckleberry Finn',
 'He is a good choice! The state and school could use fortifications and escape routes to']

### Generate multiple conclusions for development dataset:

In [70]:
sample_df = valid_df[valid_df.post.str.len() < 50].sample(10)

In [73]:
sample_df['post_concepts'] = get_wiki_concepts(sample_df['post'].tolist(), max_concepts=5)

sample_posts_conclusions = []
for i in range(5):
    control_codes = [x[i] if len(x) > i else '' for x in sample_df['post_concepts'].tolist()]
    conclusions   = generate_conclusion(sample_df['post'].tolist(), control_codes, gen_kwargs)
    sample_posts_conclusions.append(conclusions)

sample_df['post_conclusions'] = list(map(list, zip(*sample_posts_conclusions))) # transpose

In [74]:
sample_df[['title', 'post_concepts', 'post_conclusions']].head()

,title,post_concepts,post_conclusions
349404,Canadian is also an ethnicity.,"[Ethnic group, Social group]","[Ethnic group can be explained as 'CanAD', groupSocial in person i feel canadian is real, Canada canadi is a real ethnicity., i believe the existence of a ""Canada"" (or some ""Canrad ethnicity, i believe canadian is a real ethnicity and denying that canadian does not mean]"
402340,Juul is having a positive social impact and should be allowed to do lifestyle advertisement,"[Electronic cigarette, Health effect, Health risk assessment]","[Electronic cigarette makers (i.e. Juul) should be allowed to use their, Health effect Juul should be allowed to continue with lifestyle ads., Health risk assessment is biased against Juul. (i'm seeing a few examples here), I support juul's right to continue with lifestyle advertisement, I think juul is having a positive social impact and should be allowed to]"
341265,Necrophilia is a harmless fetish that shouldn’t be demonized the same way other predatory fetishes like pedophilia are,"[Social stigma, Sexual fetishism, Violent crime, Sex toy, Inventive step and non-obviousness]","[Social stigma surrounding necrophilia is inaccurate and could actually have a positive impact., Sexual fetishism should be decriminalized. (for the most part), ent crimeViolence offenders should be able to receive the same punishment/society license as, Sex toy manufacturers should be allowed to sell necrophilia, Inventive non-obvious and stepness for necrophilia (e.g]"
334234,"True Altruism Doesn't Exist, People only Do Altruistic Things in Inflate their Ego's",[],"[True Altruism doesn't exist... we are all people., True altruism does exist. It is all self reflection., You Are The Man and People Do It For Their own Eops, true Altruism does not exist, we are all programmed, True Altruism Doesn't Exist, People only Do Altruistic things.]"
302694,People that want to further regulate or bans guns to save lives should also be willing to further regulate or ban alcohol.,"[Mental health, Political corruption, Mass shooting, Logical disjunction, Common sense]","[Mental health arguments for gun control law can be based on logic., Political corruption, the fact that most of us are gun-ed is a good thing because, Mass shooting victims should not be tried like if alcohol does not., junctionLogical disencies with respect to alcohol, Common sense alcohol regulations do more harm than good (in the US)]"
